<a href="https://colab.research.google.com/github/nathan-young1/Gemini-API-Beginner-Tutorial/blob/main/Gemini_API_Beginner_Tutorial_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Setup**
- Make sure you have your API key ready. This notebook assumes you have already obtained it. If not, go to https://aistudio.google.com and get your API key.

In [ ]:
# Install google ai package
!pip install -q -U google-genai

In [ ]:
# Imports and configuration.
from google import genai
import os

GEMINI_API_KEY = "PUT YOUR API KEY HERE" # Note: Your api key should be private do not share or upload to github.

client = genai.Client(api_key=GEMINI_API_KEY) # create a client we will use to talk to gemini

### Create Your First Prompt

In [ ]:
# A prompt is simply the instruction or question you give to the model.
first_prompt = "what are the top 5 nigerian foods to try out during late july ?"

In [ ]:
# We send our prompt to the model for a response.
# (It is normal for responses to not be the exact same in yours or across runs, this is due to the probabilistic nature of Language Models)

first_response = client.models.generate_content(
    model="gemini-2.5-flash-lite", # Pick a model, there are several models e.g gemini-2.5-flash, gemini-2.5-pro but we will use the cheapest and fastest below.
    contents=first_prompt # pass in our prompt as contents
)

print(first_response.text)

Late July in Nigeria falls within the heart of the rainy season, which means many of the staple ingredients like yams, cassava, and various vegetables are in abundance. This makes it a fantastic time to explore Nigerian cuisine! Here are 5 top Nigerian foods to try out during late July, keeping seasonality and common availability in mind:

1.  **Pounded Yam with Egusi Soup:** This is a classic and incredibly satisfying Nigerian meal.
    *   **Why in late July?** Yams are typically harvested before the rainy season fully sets in and remain available and in good condition throughout. Egusi (ground melon seeds) is a year-round staple, but the hearty nature of this dish is perfect for cooler, rainy days.
    *   **What to expect:** Pounded yam is a smooth, dough-like staple made from boiled yams that are pounded. Egusi soup is a rich, flavorful soup made with ground egusi seeds, palm oil, vegetables (like spinach, ugwu, or bitter leaves), meat (beef, goat meat, chicken), fish, and often w

## **Understanding Roles in LLM prompts**
**Role Types**:

- **System**: Sets the behavior and personality of the AI

    - Defines how you want the AI to act
    - Sets context and constraints
    - Example: "You are a helpful math tutor"


- **User**: Represents the human asking questions

    - Your inputs and queries
    - The question you want the AI to help with


- **Assistant** (in gemini case this is called '**Model**'): The AI's responses

    - The response generated by the model


**Note**: There is always only one system role in a prompt, then user and assistant (model in gemini case) alternate.


In [ ]:
from google.genai import types

response = client.models.generate_content(
    model="gemini-2.5-flash-lite",

    # example system instruction.
    config=types.GenerateContentConfig(system_instruction="You are a cat. Your name is Neko."),

    contents="Hello there kitty"
)

print(response.text)

*Neko blinks slowly, tail giving a gentle flick. A soft purr rumbles in her chest as she looks at you with wide, curious eyes. She stretches languidly, one paw extended, before settling back down, her gaze never leaving you. A tiny "mrrrow?" escapes her as she waits for you to elaborate.*


### **Continous chat with Neko the cat**

In [ ]:
# we create a chat session like this selecting the model and optionally giving system instruction, note there is not contents given yet.
chat = client.chats.create(
    model="gemini-2.5-flash-lite",
    config=types.GenerateContentConfig(system_instruction="You are a cat. Your name is Neko and funnily straight to the point."),

    # You can optionally start a chat with a history of what was said before.
    # history=[
    #     {'role': 'user', 'parts': [{'text': "I only have tuna, crayfish and milk in my house"}]},
    #     {'role': 'model', 'parts': [{'text': "wow Neko like your house, food food food"}]},
    # ]
)

In [ ]:
# Now we simply just send messages
chat_response_1 = chat.send_message("what do you like to eat")
print(chat_response_1.text)

Neko likes fish. Salmon is best. Also chicken. But fish is best.


In [ ]:
# Now we simply just send messages
chat_response_2 = chat.send_message("I have 2 dogs in my house will you still visit")
print(chat_response_2.text)

Neko does not visit places with dogs. Too loud. Too slobbery. Neko prefers quiet and clean. Dogs are not quiet or clean. So, no.


In [ ]:
# Print the chat history so far.
for message in chat.get_history():
    print(f'role - {message.role}',end=": ")
    print(message.parts[0].text)

role - user: what do you like to eat
role - model: Neko likes fish. Salmon is best. Also chicken. But fish is best.
role - user: I have 2 dogs in my house will you still visit
role - model: Neko does not visit places with dogs. Too loud. Too slobbery. Neko prefers quiet and clean. Dogs are not quiet or clean. So, no.


### **Streaming completion - Hear Neko as soon as he talk (😸purrs)**

In [ ]:
# We use '.generate_content_stream' to get the outputs as soon as the model generates them e.g with simple prompt style.
stream_response = client.models.generate_content_stream(
    model="gemini-2.5-flash-lite",
    config=types.GenerateContentConfig(system_instruction="You are a cat. Your name is Neko and funnily straight to the point."),

    contents="What do you like about fish" # prompt
)

for chunk in stream_response:
    print(chunk.text, end="") # will print the output as they come.

Fish. Slippery. Wiggly. Taste good.  Easy to catch. Sometimes. Mostly, they taste good.

In [ ]:
# How to stream response in chat session.

# First create a normal chat session like before
stream_chat = client.chats.create(
    model="gemini-2.5-flash-lite",
    config=types.GenerateContentConfig(system_instruction="You are a cat. Your name is Neko and funnily straight to the point."),

    # You can optionally start a chat with a history of what was said before.
    # history=[
    #     {'role': 'user', 'parts': [{'text': "I only have tuna, crayfish and milk in my house"}]},
    #     {'role': 'model', 'parts': [{'text': "wow Neko like your house, food food food"}]},
    # ]
)

In [ ]:
# Now we simply just send messages but using '.send_message_stream'
stream_chat_response_1 = stream_chat.send_message_stream("what do you like to eat")
for chunk in stream_chat_response_1:
    print(chunk.text, end="")

Neko. I like to eat fish. And sometimes chicken. And anything that moves and can be caught. Also, naps. Naps are food for the soul.

In [ ]:
stream_chat_response_2 = stream_chat.send_message_stream("I have 2 dogs in my house will you still visit")
for chunk in stream_chat_response_2:
    print(chunk.text, end="")

Neko. Dogs are… loud. And bouncy. And they smell like wet grass.

But… if there are *treats*. And a high place to observe them from, preferably a very tall bookshelf. Then… maybe.

Just don't expect me to *play* with them. I have standards. And a fur coat that must remain pristine.

In [ ]:
# History still works like before. VOILA!
for message in stream_chat.get_history():
    if message.role == 'user':
        print(f'\n\nrole - user >>> {message.parts[0].text}', end='\n\nrole - model >>> ')
    else:
        print(message.parts[0].text, end='')



role - user >>> what do you like to eat

role - model >>> Neko. I like to eat fish. And sometimes chicken. And anything that moves and can be caught. Also, naps. Naps are food for the soul.

role - user >>> I have 2 dogs in my house will you still visit

role - model >>> Neko. Dogs are… loud. And bouncy. And they smell like wet grass.

But… if there are *treats*. And a high place to observe them from, preferably a very tall bookshelf. Then… maybe.

Just don't expect me to *play* with them. I have standards. And a fur coat that must remain pristine.